In [1]:
import sys
from pathlib import Path
sys.path.insert(0, '../src')

from plant_care_ai.training.class_selection import get_most_popular_classes

project_root = Path.cwd().parent 
data_path = project_root / "data" / "plantnet_300K"

top_100, all_counts = get_most_popular_classes(str(data_path), top_k=100)
print(f"Top class: {top_100[0]} with {all_counts[top_100[0]]} samples")

Loaded 243916 samples, 1081 classes
Top class: 1363227 with 7208 samples
